# Homework set 6

Before you turn this problem in, make sure everything runs as expected (in the menubar, select Kernel → Restart Kernel and Run All Cells...).

Please **submit this Jupyter notebook through Canvas** no later than **Mon Dec. 11, 9:00**. **Submit the notebook file with your answers (as .ipynb file) and a pdf printout. The pdf version can be used by the teachers to provide feedback. A pdf version can be made using the save and export option in the Jupyter Lab file menu.**

Homework is in **groups of two**, and you are expected to hand in original work. Work that is copied from another group will not be accepted.

# Exercise 0
Write down the names + student ID of the people in your group.

Zijian Zhang, 14851598

Lina Xiang, 14764369

# About imports
Please import the needed packages by yourself.

In [1]:
import numpy as np


# Exercise 1
N.B.1 tentative points for each part are: 2+1.5+2+2+1.5 (and one point for free gives 10).

N.B.2 you are to implement the methods yourself.

Given a function $f$, let $T(f,a,b,m)$ denote the composite trapezoid rule with $m$ subintervals over the interval $[a,b]$. 
## (a)
Approximate the integral of $x^{-3}$ over $[a,b] = [ \frac{1}{10}, 100 ]$ by the composite trapezoid rule $T(f,a,b,m)$ for $m = 2^k$. Find the smallest $k$ such that the exact error is less than $\epsilon = 10^{-3}$. Explain the slow convergence.

In [2]:
def T(f, a, b, m):
    h = (b - a) / m  # length of subintervals
    xs = [a + i * h for i in range(1, m)]  # inner points
    Tf = sum(f(x) for x in xs) + f(a) / 2 + f(b) / 2
    return Tf * h


f = lambda x: x**-3
int_f = lambda x: -x**-2 / 2
a, b = 1 / 10, 100.
exact_value = int_f(b) - int_f(a)
epsilon = 1e-3
for k in range(100):
    m = 2**k
    exact_error = T(f, a, b, m) - exact_value
    if np.abs(exact_error) < epsilon:
        print(f'k = {k}')
        print(f'Exact error = {exact_error}')
        break

k = 18
Exact error = 0.00036306889090553796


#### **Answer:**
The slow convergence of the composite trapezoid rule in approximating the integral of \( f(x) = x^{-3} \) over the interval \([ \frac{1}{10}, 100 ]\) is primarily due to the nature of the function and the characteristics of the trapezoid rule itself. The function \( f(x) = x^{-3} \) has a singularity at \( x = 0 \), and although this point is not within the integration interval, it's very close to the lower limit \( a = \frac{1}{10} \). Near this singularity, the function's values change rapidly, posing a challenge for any numerical integration method that samples the function at discrete points. The trapezoid rule, which approximates the area under a curve by dividing the interval into subintervals and using linear approximations, struggles with functions that exhibit significant curvature or rapid changes in a small interval. Furthermore, the error in the trapezoid rule is proportional to the second derivative of the function, which in the case of \( f(x) = x^{-3} \) becomes very large as \( x \) approaches zero. This results in a larger error near the lower limit, requiring a greater number of subintervals to achieve a desired accuracy. Additionally, the non-uniformity of the function across the interval, with values decreasing rapidly as \( x \) increases, further complicates the approximation, as the simple linear approach of the trapezoid rule is less effective in capturing the function's behavior, especially near the lower limit where the function values change more drastically.


## (b)

To improve the convergence rate of the above problem, we may use an adaptive strategy, as discussed in the book and the lecture. Consider the following formulas for approximate integration
$$\begin{aligned}
I_1(f,a,b) = {}& T(f,a,b,1) \\
I_2(f,a,b) = {}& T(f,a,b,2) .
\end{aligned}$$
Show, based on the error estimates for the trapezoid rule using the Taylor series (book example 8.2) that the error in $I_2$ can be estimated by a formula of the form 
$$E_2 = C (I_1 - I_2)$$
and determine the constant $C$ (if you can't find $C$, you may take $C = 0.5$).

#### **Answer:**

## (c)
An adaptive strategy for computing the integral on an interval $[a,b]$ now is: Compute $I_2$ and $E_2$, and accept $I_2$ as an approximation when the estimated error $E_2$ is less or equal than a desired tolerance $\epsilon$.  Otherwise, apply the procedure to 
$\int_a^{\frac{b+a}{2}} f(x) \, dx$ and $\int_{\frac{b+a}{2}}^b f(x) \, dx$ with tolerances $\frac{\epsilon}{2}$.

Write a recursive python routine that implements the adaptive strategy.

Then apply this routine to the function $x^{-3}$ with $a, b, \epsilon$ as before. What is the exact error in the obtained approximation? 

In [3]:
def adaptive_trapezoid(f, a, b, epsilon):
    def T(f, a, b, m):
        h = (b - a) / m
        return h * (f(a) / 2 + sum(f(a + i * h) for i in range(1, m)) + f(b) / 2)

    I1 = T(f, a, b, 1)
    I2 = T(f, a, b, 2)
    E2 = (I1 - I2) / 3  # Using C = 1/3 based on error analysis

    if abs(E2) <= epsilon:
        return I2
    else:
        mid = (a + b) / 2
        left_integral = adaptive_trapezoid(f, a, mid, epsilon / 2)
        right_integral = adaptive_trapezoid(f, b, mid, epsilon / 2)
        return left_integral + right_integral

# Function and parameters
f = lambda x: x**-3
a, b = 1 / 10, 100.
epsilon = 1e-3

# Compute the integral using the adaptive strategy
approx_integral = adaptive_trapezoid(f, a, b, epsilon)

# Compute the exact integral for comparison
int_f = lambda x: -1 / (2 * x**2)
exact_integral = int_f(b) - int_f(a)

# Calculate the exact error
exact_error = abs(exact_integral - approx_integral)
print(f'Approximate integral: {approx_integral}')
print(f'Exact integral: {exact_integral}')
print(f'Exact error: {exact_error}')


Approximate integral: -0.01893963509394296
Exact integral: 49.99994999999999
Exact error: 50.018889635093934


## (d)
Modify the code of (c) so that the number of function evaluations is counted and that no unnecessary function evaluations are performed. Compare the number of function evaluations used in the adaptive strategy of (c) with the result of (a). 
(*Hint*: To count the number of function evaluations, you may use a global variable that is incremented by the function each time it is called.)

In [4]:
# Global variable to count function evaluations
function_evaluations = 0

def f(x):
    global function_evaluations
    function_evaluations += 1
    return x**-3

def adaptive_trapezoid(f, a, b, fa, fb, epsilon):
    global function_evaluations

    # Midpoint and function evaluation at midpoint
    mid = (a + b) / 2
    fmid = f(mid)
    
    # Trapezoid rule for one and two intervals
    I1 = (b - a) * (fa + fb) / 2
    I2 = (b - a) * (fa + 2 * fmid + fb) / 4
    E2 = (I1 - I2) / 3  # Using C = 1/3 based on error analysis

    if abs(E2) <= epsilon:
        return I2
    else:
        left_integral = adaptive_trapezoid(f, a, mid, fa, fmid, epsilon / 2)
        right_integral = adaptive_trapezoid(f, mid, b, fmid, fb, epsilon / 2)
        return left_integral + right_integral

# Initial function evaluations at the endpoints
a, b = 1 / 10, 100.
fa = f(a)
fb = f(b)

# Reset the function evaluation counter
function_evaluations = 0

# Compute the integral using the adaptive strategy
epsilon = 1e-3
approx_integral = adaptive_trapezoid(f, a, b, fa, fb, epsilon)

# Output the results
print(f'Approximate integral: {approx_integral}')
print(f'Number of function evaluations: {function_evaluations}')

# Compare with the number of evaluations in part (a)
k = 0
while 2**k < function_evaluations:
    k += 1
print(f'In part (a), for m = 2^{k}, the number of evaluations would be: {2**k + 1}')


Approximate integral: 50.00014849011892
Number of function evaluations: 9669
In part (a), for m = 2^14, the number of evaluations would be: 16385


#### **Answer:**

## (e)
In the course of executing the recursive procedure, some subintervals are refined (split in two subintervals) while others aren't as a result of the choices made by the algorithm. It turns out that the choices made by this algorithm are not always optimal. Other algorithms, that decide in a different way which subinterval needs to be refined, may be more efficient in the sense that they require less function evaluations (while using the same formulas for the approximate integral and the approximate error associated with a subinterval).

Can you explain why this is the case? Discuss briefly possible alternative approaches.


#### **Answer:**

The reason why different Strategies May Vary in Efficiency:
Non-Uniform Behavior of Functions: Many functions exhibit non-uniform behavior across their domain, with areas of rapid change (like sharp peaks or troughs) and areas of relative flatness. An algorithm that can more accurately identify and target subintervals where the function changes rapidly for refinement will generally be more efficient, as it avoids unnecessary computations in regions where the function is relatively smooth.

Error Distribution: The error in numerical integration is not uniformly distributed across the interval. Some regions may contribute significantly more to the total error than others. An optimal algorithm would allocate more computational resources (i.e., function evaluations) to regions contributing most to the error.

Alternative Approaches: